# PySpark

## 1. PySpark 核心类介绍

- **pyspark.SparkContext** : SparkContext存在于 **Driver** 中, 是Spark库的主要入口点, 它表示与Spark集群的一个连接, 其他重要的对象都要依赖它. 它可以在集群上创建RDD, accumulators和广播变量等分布式数据结构.
- **pyspark.sql.SparkSession** : SparkSession是Spark 2.0中引入的新的入口点, 它是Spark SQL, DataFrame和Dataset API的入口点. `SparkSession`是创建DataFrame和执行SQL查询的主要入口点.
- **pyspark.sql.DataFrame** : 是Spark SQL的主要抽象对象, 若干行的分布式数据, 每一行都有若干个有名字的列.  跟R/Python中的DataFrame 相像 , 有着更丰富的优化. DataFrame可以有很多种方式进行构造, 例如:  结构化数据文件, Hive的table, 外部数据库, RDD.
- **pyspark.sql.Column** :  DataFrame 的列表达.
- **pyspark.sql.Row** :  DataFrame 的行数据.

## 2. `pyspark.SparkContext` & `pyspark.sql.SparkSession`

Spark 2.20 以后 SparkSession 合并了 SQLContext 和 HiveContext, 同时支持Hive, 包括HIveSOL, Hive UDFs 的入口, 以及从Hive table中读取数据.

In [7]:
import os
os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-1.8"
# 配置hadoop路径
os.environ['HADOOP_HOME'] = "D:/hadoop-3.3.4"
# 配置python解释器
os.environ['PYSPARK_PYTHON'] = "D:/Anaconda/envs/dsml/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "D:/Anaconda/envs/dsml/python.exe"

In [8]:
from pyspark.sql import SparkSession

## 获取或者新建一个 sparkSession
spark = (SparkSession
         .builder
         .appName("Python Spark SQL basic example")
         .master("local[2]")       #spark master URL. 本地为local, “local[4]” 本地4核,
                                   # or “spark://master:7077” to run on a Spark standalone cluster
         .config("spark.executor.memory", "512m")
         .getOrCreate() )          # 获取现有的SparkSession或创建新的，避免重复创建

## 从SparkSession中获取底层的SparkContext对象
## SparkContext是Spark功能的原始入口点，用于:
#    创建RDDs
#    设置配置
#    访问集群资源
#    低级API操作
sc = spark.sparkContext


## 3. `pyspark.sql.DataFrame` : DataFrame

### 3.1. 创建DataFrame
有了SparkSession, DataFrame可以从已有的RDD, Hive table, 或者其他spark的数据源进行创建.

In [9]:
# spark is an existing SparkSession
# 从文件读取
## read.json
df = spark.read.json("C:\\Users\\86188\\Desktop\\CODE\\llm-from-scratch\\data_aug.jsonl")
df.show()
# df = spark.read.load("C:\\Users\\86188\\Desktop\\CODE\\llm-from-scratch\\data_aug.jsonl", format="json") #format: Default to 'parquet'

## read.csv
## df_csv = spark.read.csv("examples/src/main/resources/people.csv",sep=';', header= True)
## read.text
## df_txt = spark.read.text("examples/src/main/resources/people.txt")
## read.parquet
## df_parquet = spark.read.parquet("examples/src/main/resources/users.parquet")
## rdd
#  sc = spark.sparkContext
#  rdd = sc.textFile('examples/src/main/resources/people.json')
#  df_rdd1 = spark.read.json(rdd)

+----------------------+----------------------+-------------------------------------+
|                 input|           instruction|                               output|
+----------------------+----------------------+-------------------------------------+
|今天北京的天气怎么样？|今天北京的天气怎么样？|         北京今天晴，气温在25度左右。|
|如何提高英语口语能力？|如何提高英语口语能力？|可以通过多听多说、多与人交流、模仿...|
+----------------------+----------------------+-------------------------------------+



In [10]:
## 通过 python list 创建DataFrame
# ["name", "height"] 指定列名为 name 和 height
df_list = spark.createDataFrame([('Tom', 80), ('Alice', None)], ["name", "height"])
df_list.show()

+-----+------+
| name|height|
+-----+------+
|  Tom|    80|
|Alice|  NULL|
+-----+------+



In [11]:
## 通过RDD创建DataFrame, 更推荐
rdd_list = [('Alice', 1), ('Bob', 2)]
rdd = sc.parallelize(rdd_list)  # sc.parallelize(l)将Python列表转换为Spark的RDD
df_rdd = spark.createDataFrame(rdd,['name', 'age'])
df_rdd.show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
|  Bob|  2|
+-----+---+



In [12]:
## 通过定义Schema创建DataFrame

from pyspark.sql.types import StructType, StructField, StringType, IntegerType
schema = StructType([    # StructType: 定义表结构 (类似关系型数据库的表结构)
    StructField("name", StringType(), True),  # StructField: 定义每个列的属性：
    StructField("age", IntegerType(), True)])
# 使用之前创建的 rdd 和定义好的schema创建DataFrame
df_schema = spark.createDataFrame(rdd, schema)
df_schema.show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
|  Bob|  2|
+-----+---+



In [13]:
## 从Pandas DataFrame 创建 spark DataFrame
import pandas as pd
df_pandas = spark.createDataFrame(pd.DataFrame([("a", 6), ("b", 7)], columns=["name", "age"]))
df_pandas.show()

+----+---+
|name|age|
+----+---+
|   a|  6|
|   b|  7|
+----+---+



### 3.2. DataFrame 常用方法

关于DataFrame的操作, 感觉上和`pandas.DataFrame`的操作很类似, 很多时候都可以触类旁通.

<p>

Spark 的操作分为两部分, 转换(transformation) 和 执行 (action). 操作是**lazy**模式, 只有遇到 **执行操作(action)** 才会执行

<p>

基本操作:

```python
df_customers.cache() # 以列式存储在内存中
df_customers.persist() # 缓存到内存中
df_customers.unpersist() # 移除所有的blocks
df_customers.coalesce(numPartitions= 1) #返回一个有着numPartition的DataFrame
df_customers.repartition(10) ##repartitonByRange
df_customers.rdd.getNumPartitions()# 查看 partitons 个数
df_customers.columns # 查看列名
df_customers.dtypes # 返回列的数据类型
df_customers.explain() #返回物理计划，调试时应用
```


In [14]:
### 创建DataFrame : customers, products, sales
customers =  [(1,'James',21,'M'), (2, "Liz",25,"F"), (3, "John", 31, "M"),
     (4, "Jennifer", 45, "F"), (5, "Robert", 41, "M"), (6, "Sandra", 45, "F")]
df_customers = spark.createDataFrame(customers, ["cID", "name", "age", "gender"]) # list -> DF
df_customers.show()

+---+--------+---+------+
|cID|    name|age|gender|
+---+--------+---+------+
|  1|   James| 21|     M|
|  2|     Liz| 25|     F|
|  3|    John| 31|     M|
|  4|Jennifer| 45|     F|
|  5|  Robert| 41|     M|
|  6|  Sandra| 45|     F|
+---+--------+---+------+



In [15]:
products = [(1, "iPhone", 600, 400), (2, "Galaxy", 500, 400), (3, "iPad", 400, 300),
            (4, "Kindel",200,100), (5, "MacBook", 1200, 900), (6, "Dell",500, 400)]
df_products = sc.parallelize(products).toDF(["pId", "name", "price", "cost"]) # List-> RDD ->DF
df_products.show()

+---+-------+-----+----+
|pId|   name|price|cost|
+---+-------+-----+----+
|  1| iPhone|  600| 400|
|  2| Galaxy|  500| 400|
|  3|   iPad|  400| 300|
|  4| Kindel|  200| 100|
|  5|MacBook| 1200| 900|
|  6|   Dell|  500| 400|
+---+-------+-----+----+



In [19]:
sales = [("01/01/2015", "iPhone", "USA", 40000), ("01/02/2015", "iPhone", "USA", 30000),
        ("01/02/2015", "iPhone", "China", 10000), ("01/02/2015", "iPhone", "China", 5000),
        ("01/01/2015", "S6", "USA", 20000), ("01/02/2015", "S6", "USA", 10000),
        ("01/01/2015", "S6", "China", 9000), ("01/02/2015", "S6", "China", 6000)]
df_sales = spark.createDataFrame(sales, ["date", "product", "country", "revenue"])

AttributeError: 'NoneType' object has no attribute 'sc'

In [17]:
df_customers.rdd.getNumPartitions()# 查看 partitons 个数

2

#### 3.2.1 Actions

- Action 操作会触发实际的计算并返回结果 (到Driver程序或存储系统). 执行Action时, Spark会执行之前所有的Transformation操作(惰性求值).
- 减少Action操作: 避免频繁调用`collect()`等操作, 减少Driver内存压力
- Actions 特点:
    - 立即触发计算
    - 通常返回非DataFrame结果 (标量、数组或直接输出)
    - 有些操作可能将大量数据传输到 Driver (如collect())

- Common Actions

| 操作             | 描述 | 示例 |
|----------------|--|------|
| `show()`       | 显示DataFrame内容 | `df.show(10)` |
| `collect()`    | 返回所有数据到Driver | `df.collect()` |
| `count()`      | 返回行数 | `df.count()` |
| `first()`      | 返回第一行 | `df.first()` |
| `take(n)`      | 返回前n行 | `df.take(5)` |
| `head(n)`      | 同take | `df.head(5)` |
| `foreach()`      | 对每行应用函数 | `df.foreach(print)` |
| `toPandas()`      | 转为Pandas DataFrame | `df.limit(1000).toPandas()` |
| `write.save()` | 保存DataFrame | `df.write.csv("path")` |









#### 3.2.2 Transformations

Transformation 操作只是定义计算逻辑, 不会立即执行, 而是记录在"执行计划"中, 直到遇到Action才执行.

- 常规Transformation操作

| 操作              | 描述 | 示例                               |
|-----------------|------|----------------------------------|
| `select()`        | 选择列 | `df.select("name", "age")` |
| `filter()/where()` | 条件过滤 | `df.filter(df.age > 30)` |
| `withColumn()`    | 添加/替换列 | `df.withColumn("age2", df.age+1)` |
| `drop()`          | 删除列 | `df.drop("temp_col")`            |
| `limit()`         | 限制行数 | `df.limit(100)`                  |

- 聚合操作

| 操作方法        | 功能描述                 | 示例代码                                      | 说明                              |
|----------------|--------------------------|---------------------------------------------|---------------------------------|
| `groupBy()`     | 基础分组操作              | `df.groupBy("dept")`                        | 按指定列分组，返回GroupedData对象，需配合聚合函数使用 |
| `agg()`         | 聚合计算                  | `df.groupBy().agg({"salary":"avg"})`        | 可对分组后的数据执行多种聚合计算，支持字典或表达式形式     |
| `rollup()`      | 多维聚合（层次式）        | `df.rollup("region").count()`               | 生成从最细粒度到总计的多层次聚合(如地区小计→全国总计)    |
| `cube()`        | 全组合多维聚合            | `df.cube("year", "month").sum()`            | 生成所有列组合的聚合                      |

| 聚合函数       | 描述                  | 使用示例                                      |
|---------------|-----------------------|---------------------------------------------|
| `count()`     | 计数                  | `df.groupBy("dept").count()`                |
| `avg()`       | 平均值                | `df.groupBy().agg(F.avg("salary"))`         |
| `sum()`       | 求和                  | `df.rollup("region").sum("sales")`          |
| `max()`       | 最大值                | `df.cube("year").max("temperature")`        |
| `min()`       | 最小值                | `df.groupBy("team").min("score")`           |
| `collect_list()` | 收集为列表           | `df.groupBy("class").agg(F.collect_list("name"))` |


In [18]:
sc.stop()
spark.stop()